In [27]:
import folium

In [28]:
my_map = folium.Map(location=[-33.4500,-70.6667],
                   tiles='Stamen Terrain', zoom_start=12)
my_map

In [29]:
my_map.choropleth(geo_data='jsons/all.geojson')
my_map

In [30]:
my_map2 = folium.Map(location=[-33.4500,-70.6667], zoom_start=12)
my_map2

In [35]:
import geopy
from geopy.geocoders import ArcGIS  # el geocodificador
import pandas as pd
import mysql.connector





nom = ArcGIS()

con = mysql.connector.connect(
    host='localhost',
    user='root',
    password='admin',
    database='establecimientos'

)

cursor = con.cursor(buffered=True)

df = pd.read_sql('SELECT * FROM negocios', con=con)
df_lat = list(df["latitud"])
df_lon = list(df["longitud"])
df_nombre_negocio = list(df["nombre"])
df_direccion_negocio = list(df['direccion'])
df_comuna_negocio = list(df['comuna'])
df_fase_comuna_negocio = list(df['fasecomuna'])

df2 = pd.read_sql(
    'SELECT nombre_comuna, fase_comuna FROM comunas where codigo_region = 13', con=con)
df2_nombre_comuna = list(df2["nombre_comuna"])
df2_fase_comuna = list(df2["fase_comuna"])


def color_marcador(fase):
    if fase == '1':
        return 'red'
    elif fase == '2':
        return 'orange'
    elif fase == '3':
        return 'green'
    else:
        return 'blue'

my_santiago_info = pd.read_sql('SELECT * FROM comunas where codigo_region = 13', con=con)
my_santiago_info.to_csv('csvs/my_santiago_info.csv', index=False,) 

my_info = r'csvs/my_santiago_info.csv'
my_info = pd.read_csv(my_info)
mymap = folium.Map(location=[-33.40, -70.60],
                   zoom_start=12,)



fg = folium.FeatureGroup(name='Establecimientos')
for (lt, ln, nom, direccionegocio, com_negocio, fase_neg) in zip(df_lat, df_lon, df_nombre_negocio, df_direccion_negocio, df_comuna_negocio, df_fase_comuna_negocio):
    fg.add_child(folium.Marker(location=[lt, ln], popup=(nom + '\nDireccion: ' + direccionegocio + '\nComuna: ' +
                 com_negocio + '\nFase: ' + fase_neg), icon=folium.Icon(icon='info-sign', color=color_marcador(fase_neg))))



# bins = list(my_info["fase_comuna"].quantile([0,0.25,0.5,0.75,1]))
folium.Choropleth( geo_data='jsons/all.json',
    name="Fases",
    data=my_info,
    columns=['nombre_comuna', 'fase_comuna'],
    key_on="feature.properties.NOM_COM",
    fill_color='RdYlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Fase en Santiago",
    bins = 4,
    reset=True,).add_to(mymap)

mymap.add_child(fg)
mymap.add_child(folium.LayerControl())
mymap.save('MapaConDb.html')
mymap
